In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importing Libraries
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

2023-02-14 09:28:30.358189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 09:28:31.000142: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 09:28:31.000170: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 09:28:32.784114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
# Data generation for training data
train_data = ImageDataGenerator(rescale = 1./255)

In [4]:
validation_data = ImageDataGenerator(rescale=1./255)

In [5]:
# Training Dataset
train = train_data.flow_from_directory(directory ='/home/ubuntu/Desktop/sep_data_final/sep_data/train/',
                                       target_size=(224, 224),
                                       batch_size = 32,
                                       shuffle = True,
                                       class_mode ="categorical")

Found 10880 images belonging to 36 classes.


In [6]:
#validation dataset
validation = validation_data.flow_from_directory(directory ='/home/ubuntu/Desktop/sep_data_final/sep_data/val/',
                                       target_size=(224, 224),
                                       batch_size = 32,
                                       shuffle = False,
                                       class_mode ="categorical")

Found 2050 images belonging to 36 classes.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,BatchNormalization

In [8]:
res_net = keras.applications.ResNet50V2(
    weights = "imagenet",
    input_shape = (224,224,3),
    include_top = False)

res_net.trainable = False

2023-02-14 09:28:35.408107: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-14 09:28:35.408136: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-14 09:28:35.408158: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-Zako): /proc/driver/nvidia/version does not exist
2023-02-14 09:28:35.409090: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Create a sequential model
model = Sequential()

# Add ResNet model to sequential model
model.add(res_net)

# Convert the model into 1d array
model.add(Flatten())

# First fully connnected layer with dropout and batchnormalization
model.add(Dense(units = 128,activation = "relu"))
model.add(Dropout(0.20))
    
# Second fully connnected layer with dropout and batchnormalization
model.add(Dense(units = 64,activation = "relu"))
model.add(BatchNormalization())
    
# Third fully connnected layer with dropout
model.add(Dense(units = 32,activation = "relu"))
model.add(Dropout(0.20))
    
# Final output layer with softmax activation function
model.add(Dense(units = 36,activation = "softmax"))


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 128)               12845184  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                        

In [11]:
model.compile(optimizer = 'adam',loss=['categorical_crossentropy'],metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
# Early stopping to avoid overfitting
es = EarlyStopping(monitor = "accuracy",patience = 3)

In [13]:
model.fit(train, validation_data = validation, epochs = 10, callbacks = [es])

Epoch 1/10
340/340 [==============================] - 2683s 8s/step - loss: 1.0927 - accuracy: 0.7429 - val_loss: 0.3733 - val_accuracy: 0.9298
Epoch 2/10
340/340 [==============================] - 2617s 8s/step - loss: 0.2419 - accuracy: 0.9438 - val_loss: 0.2666 - val_accuracy: 0.9478
Epoch 3/10
340/340 [==============================] - 2247s 7s/step - loss: 0.1293 - accuracy: 0.9665 - val_loss: 0.2327 - val_accuracy: 0.9561
Epoch 4/10
340/340 [==============================] - 2489s 7s/step - loss: 0.0790 - accuracy: 0.9813 - val_loss: 0.2567 - val_accuracy: 0.9502
Epoch 5/10
340/340 [==============================] - 2411s 7s/step - loss: 0.0667 - accuracy: 0.9827 - val_loss: 0.2622 - val_accuracy: 0.9590
Epoch 6/10
340/340 [==============================] - 2279s 7s/step - loss: 0.0588 - accuracy: 0.9839 - val_loss: 0.2659 - val_accuracy: 0.9571
Epoch 7/10
340/340 [==============================] - 2213s 7s/step - loss: 0.0445 - accuracy: 0.9870 - val_loss: 0.2815 - val_accuracy:

In [14]:
model.save("model1.h5")